<a href="https://colab.research.google.com/github/jayesh2708/ICBP-Project/blob/main/ICBP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torchvision.models import VGG19_Weights
from PIL import Image
import matplotlib.pyplot as plt
import os
from google.colab import files  # For image upload


In [14]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [15]:
# Load and transform images
def load_image(image_path, size=512):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x[:3, :, :]),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    image = transform(image).unsqueeze(0)
    return image.to(device)

In [16]:
# Display image
def imshow(tensor, title=None):
    image = tensor.clone().detach().cpu().squeeze(0)
    image = transforms.Normalize(
        mean=[-2.118, -2.036, -1.804],
        std=[4.367, 4.464, 4.444]
    )(image)
    image = torch.clamp(image, 0, 1)
    image = transforms.ToPILImage()(image)
    if title:
        plt.title(title)
    plt.imshow(image)
    plt.axis('off')
    plt.show()


In [17]:
# VGG19 for feature extraction
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = models.vgg19(weights=VGG19_Weights.DEFAULT).features[:21].to(device).eval()
        for param in self.features.parameters():
            param.requires_grad = False

    def forward(self, x):
        features = []
        for i, layer in enumerate(self.features):
            x = layer(x)
            if i in {0, 5, 10, 19}:
                features.append(x)
        return features

In [18]:
# Gram matrix for style
def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    tensor = tensor.view(c, h * w)
    return torch.mm(tensor, tensor.t())

In [19]:
# Loss function
class StyleTransferLoss(nn.Module):
    def __init__(self, content_img, style_img, model, alpha=1, beta=1e6):
        super(StyleTransferLoss, self).__init__()
        self.vgg_model = model
        self.content_features = self.vgg_model(content_img)
        self.style_features = self.vgg_model(style_img)
        self.alpha = alpha
        self.beta = beta

    def forward(self, input_img):
        input_features = self.vgg_model(input_img)
        content_loss = torch.mean((input_features[2] - self.content_features[2])**2)

        style_loss = 0
        for inp_feat, style_feat in zip(input_features, self.style_features):
            G = gram_matrix(inp_feat)
            A = gram_matrix(style_feat)
            style_loss += torch.mean((G - A)**2)

        total_loss = self.alpha * content_loss + self.beta * style_loss
        return total_loss

In [20]:
# Save output image
def save_image(tensor, filename):
    image = tensor.clone().detach().cpu().squeeze(0)
    image = transforms.Normalize(
        mean=[-2.118, -2.036, -1.804],
        std=[4.367, 4.464, 4.444]
    )(image)
    image = torch.clamp(image, 0, 1)
    transforms.ToPILImage()(image).save(filename)
    print(f"\n✅ Saved stylized image as {filename}")

In [21]:
# Display content, style, and output images
def show_all_images(content_path, style_path, output_path):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    content = Image.open(content_path).convert('RGB')
    style = Image.open(style_path).convert('RGB')
    output = Image.open(output_path).convert('RGB')

    axs[0].imshow(content)
    axs[0].set_title("Content Image")
    axs[0].axis('off')

    axs[1].imshow(style)
    axs[1].set_title("Style Image")
    axs[1].axis('off')

    axs[2].imshow(output)
    axs[2].set_title("Stylized Output")
    axs[2].axis('off')

    plt.tight_layout()
    plt.show()

In [22]:
# Style transfer training
def run_style_transfer(content_path, style_path, output_path, num_steps=300, style_weight=1e6):
    content_img = load_image(content_path)
    style_img = load_image(style_path, size=content_img.shape[-1])
    input_img = content_img.clone().requires_grad_(True)

    vgg_model = VGG()
    loss_fn = StyleTransferLoss(content_img, style_img, vgg_model)
    optimizer = optim.Adam([input_img], lr=0.003)

    print("\nTraining started...")
    for step in range(1, num_steps+1):
        optimizer.zero_grad()
        loss = loss_fn(input_img)
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            print(f"Step {step}, Loss: {loss.item():.2f}")

    print("\nTraining complete.")
    save_image(input_img, output_path)
    imshow(input_img, title="Stylized Output")
    show_all_images(content_path, style_path, output_path)


In [23]:
# Upload files interactively in Colab
print("📤 Please upload your content and style images")
uploaded = files.upload()
print("✅ Uploaded files:", list(uploaded.keys()))



📤 Please upload your content and style images


Saving my_pic.jpg to my_pic (1).jpg
Saving oil_painting.jpg to oil_painting (1).jpg
✅ Uploaded files: ['my_pic (1).jpg', 'oil_painting (1).jpg']


In [27]:
# Example usage
# Use exact filenames from the upload step
content_path = list(uploaded.keys())[0]  # First uploaded file
style_path = list(uploaded.keys())[1]    # Second uploaded file
output_path = 'styled_output.jpg'
run_style_transfer(content_path, style_path, output_path)



Training started...


KeyboardInterrupt: 

In [ ]:
# 🔧 Install required packages
!pip install -q realesrgan basicsr gfpgan


In [ ]:
from google.colab import files
from PIL import Image

print("📤 Please upload a low-resolution image (JPG/PNG):")
uploaded = files.upload()
input_path = list(uploaded.keys())[0]


In [ ]:
from realesrgan import RealESRGAN
import torch

def upscale_image(input_path, scale=4, output_path="upscaled_output.jpg"):
    # ⚙️ Initialize model
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = RealESRGAN(device, scale=scale)
    model.load_weights(f"RealESRGAN_x{scale}.pth", download=True)

    # 🔍 Open and process the image
    image = Image.open(input_path).convert("RGB")
    sr_image = model.predict(image)

    # 💾 Save upscaled image
    sr_image.save(output_path)
    print(f"\n✅ Upscaled image saved as: {output_path}")
    return output_path


In [ ]:
import matplotlib.pyplot as plt

def show_comparison(original_path, upscaled_path):
    original = Image.open(original_path).convert("RGB")
    upscaled = Image.open(upscaled_path).convert("RGB")

    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(original)
    plt.title("🔹 Original Image")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(upscaled)
    plt.title("🔸 Upscaled Image (4x)")
    plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# ⚙️ Run super-resolution
upscaled_image_path = upscale_image(input_path, scale=4)

# 👁️‍🗨️ Show before-after comparison
show_comparison(input_path, upscaled_image_path)


In [ ]:
files.download(upscaled_image_path)
